In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

# Step 1: Make a request to the webpage URL and extract data from fourth table
url = 'https://www.chancedegol.com.br/br23.htm'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find_all('table')[7]

# Step 2: Loop through the rows of the table and extract the data from the cells
data = []
for row in table.find_all('tr'):
    cols = row.find_all('td')
    cols = [col.text.strip() for col in cols]
    data.append(cols)

# Step 3: Write the data to a CSV file
with open('dataBRA23raw.csv', 'w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerows(data)
    


In [7]:
df = pd.read_csv('dataBRA23raw.csv')
df

,Data,Mandante,Unnamed: 2,Visitante,Vitória doMandante,Empate,Vitória doVisitante
0,13/11/2022,Athletico PR,3x0,Botafogo,43.1 %,26.8 %,30.1 %
1,15/04/2023,Botafogo,2x1,São Paulo,36.9 %,27.9 %,35.2 %
2,15/04/2023,Palmeiras,2x1,Cuiabá,74.0 %,18.6 %,7.4 %
3,15/04/2023,Red Bull Bragantino,2x1,Bahia,56.7 %,23.7 %,19.6 %
4,15/04/2023,Atlético MG,1x2,Vasco,55.1 %,25.9 %,19.0 %
...,...,...,...,...,...,...,...
204,27/08/2023,Atlético MG,2x0,Santos,64.8 %,23.4 %,11.7 %
205,27/08/2023,Grêmio,3x0,Cruzeiro,52.2 %,28.7 %,19.1 %
206,27/08/2023,Fortaleza,3x1,Coritiba,73.7 %,17.5 %,8.9 %
207,27/08/2023,América MG,2x1,São Paulo,28.5 %,23.8 %,47.7 %


In [8]:
#Step 8: cleaning data frame

df.rename(columns={'Mandante':'Casa','Visitante':'Fora'}, inplace =True)
#changes dtype to datetime in first dataframe
df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
#creates a new column with the for the competition name

df = df.assign(Comp='Série A')

#separets the results of second dataframe into new columns
new = df["Unnamed: 2"].str.split("x", n = 1, expand = True)
df["GC"]= new[0] 
df["GF"]= new[1]
#creates new column with competition name

#comparing scores to display the results in the second dataframe
df.loc[df['GC'] == df['GF'], 'Res'] = 'E'
df.loc[df['GC'] > df['GF'], 'Res'] = 'VC' 
df.loc[df['GC'] < df['GF'], 'Res'] = 'VV'
# create a new column to store the season information in the second data frame
df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
df['Temp'] = df['Data'].dt.year
#organizes de columns of the sacond dataframe
new_order = ['Comp','Temp','Data','Casa','GC', 'GF','Fora','Res']
df= df.reindex(columns=new_order)



df.replace({'América MG': 'América-MG',
            'Atlético MG': 'Atlético-MG',
            'Athletico PR': 'Athletico-PR',                   
            }, inplace=True)

#drops unvalid data
df.dropna(axis=0, how='all', inplace=True)
#sort values by date
df = df.sort_values(by='Data')

#atributes a pontuation to each game
df['PC'] = df.apply(lambda x: 3 if x['Res'] == 'VC' else
                              1 if x['Res'] == 'E' else 0, axis=1)
df['PF'] = df.apply(lambda x: 3 if x['Res'] == 'VV' else
                              1 if x['Res'] == 'E' else 0, axis=1)

#atributes a number of games for each season
df['J'] = df.groupby('Temp').cumcount() + 1
#corrects any possible issue before saving as new file
df = df.dropna()
df = df.drop(index = 0)
df

C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_12120\1842171339.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_12120\1842171339.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_12120\1842171339.py:21: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
1,Série A,2023,2023-04-15,Botafogo,2,1,São Paulo,VC,3,0,1
2,Série A,2023,2023-04-15,Palmeiras,2,1,Cuiabá,VC,3,0,2
3,Série A,2023,2023-04-15,Red Bull Bragantino,2,1,Bahia,VC,3,0,3
4,Série A,2023,2023-04-15,Atlético-MG,1,2,Vasco,VV,0,3,4
5,Série A,2023,2023-04-15,Athletico-PR,2,0,Goiás,VC,3,0,5
...,...,...,...,...,...,...,...,...,...,...,...
207,Série A,2023,2023-08-27,América-MG,2,1,São Paulo,VC,3,0,204
202,Série A,2023,2023-08-27,Botafogo,3,0,Bahia,VC,3,0,205
201,Série A,2023,2023-08-27,Athletico-PR,2,2,Fluminense,E,1,1,206
203,Série A,2023,2023-08-27,Red Bull Bragantino,2,0,Cuiabá,VC,3,0,207


In [9]:

df.to_csv('data/dataBRA23.csv', index=False)
df

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
1,Série A,2023,2023-04-15,Botafogo,2,1,São Paulo,VC,3,0,1
2,Série A,2023,2023-04-15,Palmeiras,2,1,Cuiabá,VC,3,0,2
3,Série A,2023,2023-04-15,Red Bull Bragantino,2,1,Bahia,VC,3,0,3
4,Série A,2023,2023-04-15,Atlético-MG,1,2,Vasco,VV,0,3,4
5,Série A,2023,2023-04-15,Athletico-PR,2,0,Goiás,VC,3,0,5
...,...,...,...,...,...,...,...,...,...,...,...
207,Série A,2023,2023-08-27,América-MG,2,1,São Paulo,VC,3,0,204
202,Série A,2023,2023-08-27,Botafogo,3,0,Bahia,VC,3,0,205
201,Série A,2023,2023-08-27,Athletico-PR,2,2,Fluminense,E,1,1,206
203,Série A,2023,2023-08-27,Red Bull Bragantino,2,0,Cuiabá,VC,3,0,207


In [10]:
df.tail(10)

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
199,Série A,2023,2023-08-26,Flamengo,0,0,Internacional,E,1,1,199
200,Série A,2023,2023-08-26,Corinthians,1,1,Goiás,E,1,1,200
206,Série A,2023,2023-08-27,Fortaleza,3,1,Coritiba,VC,3,0,201
205,Série A,2023,2023-08-27,Grêmio,3,0,Cruzeiro,VC,3,0,202
204,Série A,2023,2023-08-27,Atlético-MG,2,0,Santos,VC,3,0,203
207,Série A,2023,2023-08-27,América-MG,2,1,São Paulo,VC,3,0,204
202,Série A,2023,2023-08-27,Botafogo,3,0,Bahia,VC,3,0,205
201,Série A,2023,2023-08-27,Athletico-PR,2,2,Fluminense,E,1,1,206
203,Série A,2023,2023-08-27,Red Bull Bragantino,2,0,Cuiabá,VC,3,0,207
208,Série A,2023,2023-08-28,Palmeiras,1,0,Vasco,VC,3,0,208
